In [475]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

The data used in this analysis is provided by [TSA]('https://www.tsa.gov/coronavirus/passenger-throughput') for daily flight passengers, and [Our World in Data]('https://ourworldindata.org/coronavirus') for all COVID-19 coronavirus confirmed cases by location. 

In [476]:
df = pd.read_csv('owid-covid-data.csv') #read in COVID data

usa = df[df['location'] == 'United States'] #looking at US data to match TSA daily flight data
usa.reset_index(inplace=True, drop=True)

In [477]:
usa = usa[['date', 'location', 'total_cases']] #only really care about total cases

In [478]:
usa.head() 

,date,location,total_cases
0,2019-12-31,United States,0.0
1,2020-01-01,United States,0.0
2,2020-01-02,United States,0.0
3,2020-01-03,United States,0.0
4,2020-01-04,United States,0.0


In [479]:
# Libraries to parse html table from TSA webpage
import requests
import lxml.html as lh 

In [480]:
url='https://www.tsa.gov/coronavirus/passenger-throughput'
#Create a handle, page, to handle the contents of the website
page = requests.get(url)
#Store the contents of the website under doc
doc = lh.fromstring(page.content)
#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

In [481]:
tr_elements = doc.xpath('//tr')
#Create empty list
col=[]
i=0
#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print ('%d:%s'%(i,name))
    col.append((name,[]))

1:Date
2:Total Traveler Throughput
3:Total Traveler Throughput
			(1 Year Ago - Same Weekday)


In [482]:
#Since out first row is the header, data is stored on the second row onwards
for j in range(1,len(tr_elements)):
    T=tr_elements[j]
    i=0
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

In [483]:
Dict = {title:column for (title,column) in col}
tsa = pd.DataFrame(Dict)

In [484]:
tsa = tsa.iloc[::-1] #reverse order of rows to match usa

tsa.head()

tsa = tsa.reset_index(drop=True)

In [485]:
tsa

,Date,Total Traveler Throughput,Total Traveler Throughput\n\t\t\t(1 Year Ago - Same Weekday)
0,3/1/2020,"2,280,522","2,301,439"
1,3/2/2020,"2,089,641","2,257,920"
2,3/3/2020,"1,736,393","1,979,558"
3,3/4/2020,"1,877,401","2,143,619"
4,3/5/2020,"2,130,015","2,402,692"
...,...,...,...
137,7/16/2020,"706,164","2,716,828"
138,7/17/2020,"720,378","2,776,960"
139,7/18/2020,"646,654","2,396,462"
140,7/19/2020,"747,422","2,727,355"


In [486]:
#Drop rows before 3/1/2020 in usa dataset
# usa.reset_index(inplace=True)

In [487]:
usa[usa['date']=='2020-03-01'] #index 61

,date,location,total_cases
61,2020-03-01,United States,69.0


In [488]:
usa = usa[61:]

In [489]:
usa = usa.reset_index(drop=True)

In [490]:
# usa = usa.set_index('date') #set date to be index

In [491]:
#Concatinate two dataframes together

data_final = pd.concat([usa, tsa], axis=1)

In [492]:
data_final

,date,location,total_cases,Date,Total Traveler Throughput,Total Traveler Throughput\n\t\t\t(1 Year Ago - Same Weekday)
0,2020-03-01,United States,69.0,3/1/2020,"2,280,522","2,301,439"
1,2020-03-02,United States,89.0,3/2/2020,"2,089,641","2,257,920"
2,2020-03-03,United States,103.0,3/3/2020,"1,736,393","1,979,558"
3,2020-03-04,United States,125.0,3/4/2020,"1,877,401","2,143,619"
4,2020-03-05,United States,159.0,3/5/2020,"2,130,015","2,402,692"
...,...,...,...,...,...,...
138,2020-07-17,United States,3576221.0,7/17/2020,"720,378","2,776,960"
139,2020-07-18,United States,3647715.0,7/18/2020,"646,654","2,396,462"
140,2020-07-19,United States,3711464.0,7/19/2020,"747,422","2,727,355"
141,2020-07-20,United States,3773260.0,7/20/2020,"695,330","2,635,312"


In [494]:
data_final = data_final.drop('Date', axis=1)